#### <u>**Imported Packages**</u>

In [63]:
%pip install wavio

Note: you may need to restart the kernel to use updated packages.


In [64]:
import wavio
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import read, write
import math
import random
import scipy.interpolate as interpolate
from typing import List, Dict, Tuple

#### **<u>Type Hinting</u>**

In [6]:
Samp = int
Sec = float
Arr = np.ndarray

#### <u>**Audio Processing**</u>

In [49]:
class AudioProcessing:
    def __init__(self, wav_file: str) -> None:
        fr = wavio.read(wav_file)
        self.data = fr.data
        self.samp_rate = fr.rate
        self.data_len = self.data.shape[0]


    def cut_by_seconds(self, start: Sec, end: Sec) -> None:
        self.update_data(self.data[start * self.samp_rate : end * self.samp_rate])
        
    def cut_by_samps(self, start: Samp, end: Samp) -> None:
        self.update_data(self.data[start:end])

    def reverse_data(self) -> None:
        self.update_data(self.data[::-1])

    def insert_glitch(self, start: Sec, dur: Sec, glitch_samps: Samp) -> None:
        """Insert a glitch in the specified second of a wav file extending the time of the file."""
        repeated_glitch_data = self.generate_glitch_from_prev_samps(start, dur, glitch_samps)

        glitch_start: Samp = round(start * self.samp_rate)
        glitched_data: Arr = np.insert(self.data, glitch_start, repeated_glitch_data, axis=0)

        self.update_data(glitched_data)
        #TODO
        #Manage unbounded seconds errors ("Can't select second {second} on a {duration} second audio")
        #Manage glitched_samps taken from the start (what is suposed to happen?).


    def corrupt_with_glitch(self, start: Sec, dur: Sec, glitch_samps: Samp) -> None:
        """Corrupt a wav file with glitch that replaces the content at a specified second and
        duration. If the glitch end surpasses the duration file, the wav will be extended."""
        repeated_glitch_data = self.generate_glitch_from_prev_samps(start, dur, glitch_samps)

        glitch_start: Samp = round(start * self.samp_rate)
        glitch_duration: Samp = round(dur * self.samp_rate)
        glitch_end: Samp = glitch_start + glitch_duration

        if self.data_len < glitch_end:
            glitched_data = np.append(self.data[:glitch_start], repeated_glitch_data, axis=0)
        else:
            glitched_data = np.append(self.data[0:glitch_start], repeated_glitch_data, axis=0)
            glitched_data = np.append(glitched_data[0:glitch_end], self.data[glitch_end:], axis=0)

        self.update_data(glitched_data)


    def scramble_data_by_bpm(self, bpm: int) -> None:
        """..."""
        samples_per_piece: Samp = round((self.samp_rate * 60) / bpm)
        reordered = self.scramble_pieces(samples_per_piece)
        self.update_data(reordered)


    def scramble_data_by_n_pieces(self, n_pieces: int) -> None:
        """Cut the data into several equal parts and reorder it randomly."""
        samples_per_piece: Samp = self.data_len//n_pieces
        reordered = self.scramble_pieces(samples_per_piece)
        self.update_data(reordered)


    def add_noise(self, noise: int = 1000) -> None:
        min_value = np.iinfo('int16').min
        max_value = np.iinfo('int16').max

        numbers = np.random.randint(0, 2, (self.data_len, 2), dtype='int32')
        numbers[numbers==0] = -noise
        numbers[numbers==1] = noise

        processed = self.data + numbers

        processed[processed>max_value] = processed[processed>max_value] - noise*2
        processed[processed<min_value] = processed[processed<min_value] + noise*2
        processed = processed.astype('int16')

        self.update_data(processed)

    def change_speed(self, rate: float) -> None:
        """Change speed and pitch of wav a file"""
        left_channel_data = self.data[:, 0]
        right_channel_data = self.data[:, 1]

        samps = np.arange(0, self.data_len, 1)

        left_channel_func = interpolate.interp1d(samps, left_channel_data, 'linear', fill_value='extrapolate')
        right_channel_func = interpolate.interp1d(samps, right_channel_data, 'linear', fill_value='extrapolate')

        new_samples = np.arange(0, self.data_len, rate)
        new_left_channel_data = left_channel_func(new_samples).reshape(new_samples.shape[0],1)
        new_right_channel_data = right_channel_func(new_samples).reshape(new_samples.shape[0],1)

        processed = np.concatenate((new_left_channel_data, new_right_channel_data), axis=1)
        processed = processed.astype('int16')

        self.update_data(processed)


    def scramble_pieces(self, samps_per_piece) -> Arr:
        pieces_ordered: list[(Samp, Samp)] = self.cut_in_pieces(samps_per_piece)
        pieces_shuffled: list[(Samp, Samp)] = list(pieces_ordered)
        random.shuffle(pieces_shuffled)

        reordered: Arr = np.zeros(shape=self.data.shape, dtype='int16')


        for ordered, shuffled in zip(pieces_ordered, pieces_shuffled):
            reordered[ordered[0]:ordered[1], :] = self.data[shuffled[0]:shuffled[1], :]

        self.update_data(self.data)
        return reordered


    def change_left_right_channels(self, samps_per_piece):
        # pieces_ordered: list[(Samp, Samp)] = self.cut_in_pieces(samps_per_piece)

        processed: Arr = np.zeros(shape=self.data.shape, dtype='int16')

        # for i, tup in enumerate(pieces_ordered):
        #     if i%2 == 1:
        #         processed[tup[0]:tup[1], 0] = self.data[tup[0]:tup[1], 1]
        #         processed[tup[0]:tup[1], 1] = self.data[tup[0]:tup[1], 0]
        #     else:
        #         processed[tup[0]:tup[1], :] = self.data[tup[0]:tup[1], :]
        # processed = self.data

        b = (np.sin(np.linspace(0, 360, 22050) * np.pi / 180. ) + 1) / 4
        b = np.tile(b, 460)[:self.data_len]
        c = (np.sin(np.linspace(180, 540, 22050) * np.pi / 180. ) + 1) / 4
        c = np.tile(c, 460)[:self.data_len]

        processed[:, 0] = self.data[:, 0] * c + self.data[:, 1] * b
        processed[:, 1] = self.data[:, 1] * c + self.data[:, 0] * b

        processed[:, 0] = self.data[:, 0] - self.data[:, 1]
        processed[:, 1] = self.data[:, 1] - self.data[:, 0]

        processed = processed.astype('int16')

        self.update_data(processed)


        # print(processed[:, 0])
        # self.update_data()

    def generate_glitch_from_prev_samps(self, start: Sec, dur: Sec, glitch_samps: Samp) -> Arr:
        """Generate a glitch (loop) from the samples of a sound file starting at a specified
        second and duration"""
        glitch_start: Samp = round(start * self.samp_rate)
        glitch_duration: Samp = round(dur * self.samp_rate)

        glitch_data: Arr = self.data[glitch_start - glitch_samps : glitch_start]
        n_repetitions: int = math.ceil(glitch_duration/glitch_data.shape[0])

        repeated_glitch_data: Arr = np.tile(glitch_data, (n_repetitions, 1))
        repeated_glitch_data: Arr = repeated_glitch_data[0:glitch_duration]

        return repeated_glitch_data


    def cut_in_pieces(self, samps_per_piece):
        sample_cuts: list[Samp] = [samp_cut for samp_cut in range(0, self.data_len+1, samps_per_piece)]
        n_pieces: int = len(sample_cuts)-1

        pieces_ordered: list[(Samp, Samp)] = []
        for i, elem in enumerate(sample_cuts):
            if i+1 < len(sample_cuts):
                pieces_ordered.append((elem, sample_cuts[i+1]))

        return pieces_ordered


    def update_data(self, new_data: Arr) -> None:
        self.data = new_data
        self.data_len = self.data.shape[0]


    def plot_data(self):
        """Plots the left channel of a wav file."""
        _time = np.arange(len(self.data))/self.samp_rate

        plt.figure(figsize=(8, 6), dpi=80)
        plt.plot(_time, self.data[:, 0])
        plt.xlabel("Seconds")
        plt.ylabel("Intensity")
        plt.show()


    def get_info(self) -> None:
        print(f"duration in samples: {self.data_len}")
        print(f"duration in seconds: {round(self.data_len / self.samp_rate, 3)}")


    def write_wav(self, name="output.wav") -> None:
        wavio.write(name, self.data, self.samp_rate)

#### <u>**Main Program**</u>

In [62]:
if __name__ == "__main__":
    file_name = "Entry of the Gladiators (snippet).wav" # write the wav file name between the quotes
    # file_name = "Entry of the Gladiators (extract).wav" # write the wav file name between the quotes
    WAV_FILE = AudioProcessing("resources/" + file_name)
    WAV_FILE.cut_by_seconds(10, 25)
    WAV_FILE.change_speed(1.2)
    WAV_FILE.corrupt_with_glitch(2.5, 0.3, 4500)
    WAV_FILE.corrupt_with_glitch(3.2, 0.2, 3400)
    WAV_FILE.corrupt_with_glitch(4.5, 0.2, 2000)
    WAV_FILE.corrupt_with_glitch(6, 0.2, 2200)
    WAV_FILE.corrupt_with_glitch(7.2, 0.4, 3200)
    WAV_FILE.corrupt_with_glitch(8, 0.1, 3200)
    WAV_FILE.corrupt_with_glitch(9, 3.2, 6200)
    WAV_FILE.write_wav()
    WAV_FILE.get_info()

duration in samples: 551250
duration in seconds: 12.5
